In [1]:
import os
import copy
import json
import bert_score
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer

/home/gridsan/ssarangerel/.conda/envs/llm-prompting/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
path = '/home/gridsan/ssarangerel/nlp/llm-prompting/model_weights'
model_checkpoints = {"flan-t5" : f'{path}/flan/flan-t5-base',
                     "llama": f'{path}/shearad_llama/v2_7',
                     "vicuna": f'{path}/vicuna-7b',
                    "mistral": f'{path}/mistral'}

from transformers import AutoTokenizer, AutoModelForCausalLM

CHOSEN_MODEL = "llama"

TOKENIZER = AutoTokenizer.from_pretrained(model_checkpoints[CHOSEN_MODEL])

if CHOSEN_MODEL in ["flan-t5"]:
    LLM = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints[CHOSEN_MODEL])
elif CHOSEN_MODEL in ["vicuna", "alpaca", "mistral", "llama"]:
    LLM = AutoModelForCausalLM.from_pretrained(model_checkpoints[CHOSEN_MODEL])
else:
    raise notImplementedError()

# BERT_SCORER = bert_score.BERTScorer(f'{path}/bert/deberta-xlarge-mnli', num_layers = 10) #num_layers ?? 
# BERT_SCORER

#　download sentencepiece and protobuf 

Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.49s/it]


In [7]:
LLM

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2560, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm(